In [169]:
#IMPORTS : 


import time
import re 

from functools import reduce
from datetime import datetime as dt

import yfinance as yf 

import numpy as np 
import pandas as pd 
import networkx as nx
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler

import dash 
from dash import Dash 
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
from dash import dcc, html, callback, callback_context, Input, Output, State

In [170]:
# Assuming you want to keep a main dataframe to store or update data
main_df = pd.DataFrame()
available_dates = main_df.index.tolist()


### Global Vars

In [171]:
# Dictionary to record : Asset Classes & tickers within 

initializer = True 

dict_classes = {
    'Equity' : [],
    'Commodities' : [],
    'Fixed Income' : [],
    'Forex' : [],
    'Index' : []
}

dict_sectors = {
    'Financials' : [],
    'Technology' : [],
    'Industrial' : [],
    'Consumer' : [],
    'Utilities' : [],
    'Other' : []
}

dict_commo = {
    'Agriculture' : [],
    'Precious Metals' : [],
    'Energy' : [],
    'Industrials' : []
}

list_tickers = []

## UTILS

In [172]:
#PAGE 1 : HOME 

def reset_global_vars():
    global initializer, dict_classes, dict_sectors, dict_commo, list_tickers
    
    initializer = True 
    
    dict_classes = {
    'Equity' : [],
    'Commodities' : [],
    'Fixed Income' : [],
    'Forex' : [],
    'Index' : []
    }

    dict_sectors = {
        'Financials' : [],
        'Technology' : [],
        'Industrial' : [],
        'Consumer' : [],
        'Utilities' : [],
        'Other' : []
    }

    dict_commo = {
        'Agriculture' : [],
        'Precious Metals' : [],
        'Energy' : [],
        'Industrials' : []
    }

    list_tickers = []
    
def scaled_df(df):

    scaler = MinMaxScaler()

    scaled_data = scaler.fit_transform(df)

    scaled_df = pd.DataFrame(scaled_data, columns=df.columns, index=df.index)

    return scaled_df


def get_logreturn(df_price_col):
    return np.log(df_price_col / df_price_col.shift(1)) * 100 


def minmax_scaler(df, spread = 1):
    min_df = df.min()
    max_df = df.max() 
    
    df = df.apply(lambda x : ((x-min_df) / (max_df - min_df)) / spread)
    
    return df 


def data_loader(ticker_name, start, end, interval='1d'):
    ticker = ticker_name 
    try : 
        df = yf.download(ticker, start=start, end=end)
        
    
        if len(df) != 0: 
            date_column = df.index
            df = df.sort_index()

            #Daily price normalized to the total volume of the dataset
            df['price_volume'] = df['Adj Close'] * df['Volume']
            df['Norm_PV'] = (df['price_volume'] / df['Volume'].sum()) #Norm_PV = Normalized Price Volume

            #Renaming (for merging purposes):
            title_1 = ticker + ': Adj Close'
            title_2 = ticker + ': Norm_PV'
            title_3 = ticker + ': Log-Returns'

            #Log-returns
            df[title_3] = get_logreturn(df['Adj Close']) #Results are in %
            df.drop(columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'price_volume'], axis = 1, inplace = True)

            df.rename(columns={'Close': title_1,
                                    'Norm_PV': title_2,
                                   }, inplace=True)
            pd.to_datetime(df.index, format='%Y-%m-%d')

        return df 
    
    except Exception as e: 
        return e



def data_loader_format_all(ticker, start, end, interval='1d'):
    global main_df, list_tickers
    
    main_df = pd.DataFrame()

    temp = list_tickers 
    temp.append(ticker)
    temp = set(temp)
    
    for i, tick in enumerate(temp):             
        if i == 0: 
            main_df = data_loader(tick, start, end)
            continue
        df = data_loader(tick, start, end)
        main_df = pd.merge(main_df, df, left_index=True, right_index=True, how='inner')

    return "All ticker frames updated." 


def curve_plotter(df, mode='price', scaled=False):
    global list_tickers 
    if mode == 'return':
        scaled=False
        suffix='Returns'
    else: 
        suffix='Adj Close'
        
    matching_columns = [col for col in df.columns if col.endswith(suffix)]
    
    temp_df = pd.DataFrame()

    df.drop(matching_columns, axis=1, inplace=True)                    
    

In [173]:
#PAGE 2 : ANALYTICS 


def relative_change(corr1, corr2):
    range_corr = 2
    rel_range_change = ((corr2 - corr1) / range_corr) * 100
    
    return rel_range_change        



def rolling_corr(df, ref_date, span):
    try:
        position = df.index.get_loc(ref_date) + 1
        start_position = position - span
        filtered_df = df.iloc[start_position:position]
        corr_matrix = filtered_df.corr()

        return corr_matrix.round(2)

    except KeyError as err: 
        print(f'Error due to wrong date/span input: {err}, Date : {ref_date}, Span : {span}.')
        print(f'Recall date range of input dataframe: {df.index[0], df.index[-1]}')

    

def matrix_difference(matrix1, matrix2):
    # Check if the matrices have the same shape
    if matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
    
    # Initialize an empty matrix to store the differences
    rows = matrix1.shape[0]
    columns = matrix1.shape[1]
    result_matrix = pd.DataFrame(np.zeros((rows, columns)))
    result_matrix.columns = list_tickers
    
    # Iterate through the rows and columns of the matrices
    for i in range(rows):
        for j in range(columns):
            perct_change = relative_change(matrix1.iloc[i, j], matrix2.iloc[i, j])
            result_matrix.iloc[i, j] = perct_change       
    return result_matrix



def matrix_difference_qual(matrix1, matrix2, heatmap=True):
    category_map = None

    # Check if the matrices have the same shape
    if matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
    
    # Initialize an empty matrix to store the differences
    rows = matrix1.shape[0]
    columns = matrix1.shape[1]
    result_matrix = pd.DataFrame(np.zeros((rows, columns)))
    result_matrix.columns = list_tickers
    
    # Iterate through the rows and columns of the matrices
    for i in range(rows):
        for j in range(columns):
            # Perform differentiation based on the values of the elements
            
            if matrix1.iloc[i, j] == 0 or matrix2.iloc[i, j] == 0:
                if matrix1.iloc[i, j] < 0:
                    matrix2.iloc[i,j] = -0.00001
                elif matrix1.iloc[i,j] > 0: 
                    matrix2.iloc[i,j] = 0.00001
                elif matrix2.iloc[i,j] < 0: 
                    matrix1.iloc[i,j] = 0.00001
                elif matrix2.iloc[i,j] > 0: 
                    matrix1.iloc[i,j] = -0.00001
                    
            if matrix1.iloc[i, j] < 0 and matrix2.iloc[i, j] < 0:
                if matrix2.iloc[i, j] > matrix1.iloc[i, j]:
                    result_matrix.iloc[i, j] = 'Neg Stronger'
                else:
                    result_matrix.iloc[i, j] = 'Neg Weaker'
            elif matrix1.iloc[i, j] > 0 and matrix2.iloc[i, j] > 0:
                if matrix2.iloc[i, j] > matrix1.iloc[i, j]:
                    result_matrix.iloc[i, j] = 'Pos Stronger'
                else:
                    result_matrix.iloc[i, j] = 'Pos Weaker'
            elif matrix1.iloc[i, j] > 0 and matrix2.iloc[i, j] < 0:
                result_matrix.iloc[i, j] = 'Neg Stronger'
            elif matrix1.iloc[i, j] < 0 and matrix2.iloc[i, j] > 0:
                result_matrix.iloc[i, j] = 'Pos Stronger'
            elif 0.95 <= matrix1.iloc[i, j] / matrix2.iloc[i, j] <= 1.05:
                result_matrix.iloc[i, j] = 'UNCH'
            else: 
                print(matrix1.iloc[i, j], matrix2.iloc[i,j])
                print(list_tickers[i], list_tickers[j])
                
    if heatmap: 
        category_map = {'Neg Stronger': -10, 'Neg Weaker': -5, 
                        'Pos Stronger': 10, 'Pos Weaker': 5, 
                        'UNCH': 0}
        df_numeric = result_matrix.applymap(lambda x: category_map[x])
        df_numeric.index = list_tickers
        
        return df_numeric

    else: 
        print("Can't return a heatmap - Categ Variables of String Type")
        return result_matrix

    

def rolling_corr_difference(df, ref_date, span):
    
    # Calculate correlation matrix for the current span
    corr_matrix_current = rolling_corr(df, ref_date, span)
    
    # Get the previous corr matrix's ref_date
    index_position = df.index.get_loc(ref_date)
    
    # Previous corr matrix's ref index position is max(0, index_position - span)
    temp_index_position = index_position - span 
    if temp_index_position < 0: 
        print(f'Period for Previous Corr Matrix calculation out of bound. Setting reference date to {df.index[0]}')
        span = index_position

    new_index_position = df.index.get_loc(df.index[temp_index_position])
    new_ref_date = df.index[new_index_position]

    # Calculate correlation matrix for the previous span
    corr_matrix_prev = rolling_corr(df, new_ref_date, span)
    
    # Calculate the difference between correlation matrices
    corr_diff = matrix_difference(corr_matrix_prev, corr_matrix_current)
    corr_diff_qual = matrix_difference_qual(corr_matrix_prev, corr_matrix_current)
    
    column_names = df.columns.tolist()
    
    #MASKING FOR HALF HEAT MAPs
    mask = np.triu(np.ones_like(corr_diff, dtype=bool))
    corr_diff_masked = np.where(mask, None, corr_diff)  # Replace upper triangular part with None
    
    mask_qual = np.triu(np.ones_like(corr_diff_qual, dtype=bool))
    corr_diff_qual_masked = np.where(mask_qual, None, corr_diff_qual)  # Replace upper triangular part with None

    
    # Plotting heatmap for relative range percentage change
    fig_relative = go.Figure(data=go.Heatmap(z=corr_diff_masked, colorscale='RdYlGn',
                                             x=column_names, y=column_names))
    fig_relative.update_layout(title=f'Relative Range Percentage Change of Rolling Correlations between Assets Log Returns, {span} freq periods.',
                               xaxis_title='Assets', yaxis_title='Assets',                           
                               plot_bgcolor='white',  paper_bgcolor='white')

    # Assuming `corr_diff_qual` is plotted here instead
    fig_directional = go.Figure(data=go.Heatmap(z=corr_diff_qual_masked, colorscale='bluered',
                                                x=column_names, y=column_names))
    fig_directional.update_layout(title=f'Directional Difference between Rolling Correlation Matrices of Assets Log Returns, {span} freq periods.',
                                  xaxis_title='Assets', yaxis_title='Assets',
                                  plot_bgcolor='white',  paper_bgcolor='white')
    # Customizing the colorbar tick labels
    fig_directional.update_traces(colorbar_tickvals=[-10, -5, 0, 5, 10],
                  colorbar_ticktext=['Negative Stronger', 'Negative Weaker', 'UNCH', 'Positive Weaker', 'Positive Stronger'])
    
    fig_relative.update_layout(
    width=900,  # Adjust width
    height=900,  # Adjust height
    title=f'Relative Range Percentage Change of Rolling Correlations between Assets Log Returns, {span} freq periods.',
    xaxis_title='Assets',
    yaxis_title='Assets',
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis={'autorange': True, 'tickangle': 45},  # Rotate x-axis labels to prevent overlap
    yaxis={'autorange': True}
    )

    fig_directional.update_layout(
        width=900,  # Adjust width
        height=900,  # Adjust height
        title=f'Directional Difference between Rolling Correlation Matrices of Assets Log Returns, {span} freq periods.',
        xaxis_title='Assets',
        yaxis_title='Assets',
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis={'autorange': True, 'tickangle': 45},  # Rotate x-axis labels to prevent overlap
        yaxis={'autorange': True}
    )
    return fig_relative, fig_directional



In [174]:
def graph_net(df, ref_date, corr_threshold, span):
    global dict_classes
    corr_matrix = rolling_corr(df, ref_date, span)
    
    # Initialize the graph
    G = nx.Graph()
    
    # Ticker categories and their colors
    ticker_categs = dict_classes
    
    colors = {
        'Equity': 'yellow',
        'Index': 'blue',
        'Fixed Income': 'red',
        'Commodities': 'orange',
        'Forex': 'green'
    }
    
    # Node colors based on their category
    node_colors = {}
    for category, nodes in ticker_categs.items():
        for node in nodes:
            node_colors[node] = colors[category]
    
    # Determine which nodes should be included based on the correlation threshold
    nodes_to_include = set()
    for i in corr_matrix.columns:
        for j in corr_matrix.columns:
            if i != j and abs(corr_matrix.loc[i, j]) > corr_threshold:
                nodes_to_include.add(i)
                nodes_to_include.add(j)
    
    # Only add nodes that are part of an edge meeting the threshold
    for node in nodes_to_include:
        G.add_node(node, color=node_colors.get(node, 'grey'))
    
    # Add edges to the graph based on correlation
    for i in corr_matrix.columns:
        for j in corr_matrix.columns:
            if i != j:  # Ensure we don't compare the same stock to itself
                corr = corr_matrix.loc[i, j]
                if abs(corr) > corr_threshold:  # Check if the correlation meets the threshold
                    # Add an edge with color based on the sign of the correlation
                    G.add_edge(i, j, weight=corr, color='green' if corr > 0 else 'red')

    # Assuming 'G' is your original graph with 'weight' attributes holding the correlations
    H = G.copy()

    # Update edge weights in H to be absolute values of the original weights
    for u, v, d in H.edges(data=True):
        d['weight'] = abs(d['weight'])


    # Assuming H is your graph for layout and G contains original correlation weights
    pos = nx.kamada_kawai_layout(H)                    
    

    # Initialize the figure once, before the loop
    fig = go.Figure()

    # For edges, create individual traces within the loop
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        corr_value = edge[2]['weight']

        # Determine the color based on the correlation value
        edge_color = 'green' if corr_value > 0 else 'red'

        # Create an individual trace for this edge
        edge_trace = go.Scatter(
            x=[x0, x1, None], 
            y=[y0, y1, None],
            line=dict(width=0.5, color=edge_color),
            mode='lines',
            hoverinfo='none',
            showlegend=False# No hover info for the line itself
        )
        fig.add_trace(edge_trace)

        # Invisible marker at the midpoint for hover text
        midpoint_trace = go.Scatter(
            x=[(x0 + x1) / 2],
            y=[(y0 + y1) / 2],
            text=[f'{edge[0]}-{edge[1]}: {corr_value:.2f}'],
            mode='markers',
            hoverinfo='text',
            marker=dict(size=0.1, color='rgba(0,0,0,0)'),  # Make the marker virtually invisible
            showlegend=False
        )
        fig.add_trace(midpoint_trace)

        
    # Track which categories have been added to the legend
    added_categories = set()
    for node in G.nodes():
        x, y = pos[node]
        category = None
        for categ, members in ticker_categs.items():
            if node in members:
                category = categ
                break
        if category and category not in added_categories:
            # Add a representative node for this category to the legend
            fig.add_trace(go.Scatter(
                x=[x], y=[y],
                mode='markers+text',
                marker=dict(color=colors[category], size=10),
                name=category,  # This sets the legend entry,
                hoverinfo='none'
            ))
            added_categories.add(category)

    # Add node trace after all edge traces have been added
    node_x = []
    node_y = []
    node_text = []
    node_marker_colors = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_marker_colors.append(G.nodes[node]['color'])

    node_trace = go.Scatter(
        x=node_x, y=node_y, text=node_text, mode='markers+text', hoverinfo='none',
        marker=dict(showscale=False, color=node_marker_colors, size=20, line_width=2),
        textposition="bottom center", showlegend=True
    )

    fig.add_trace(node_trace)

    # Set the layout for the figure
    fig.update_layout(
        showlegend=True,
        hovermode='closest',
        margin=dict(b=0,l=0,r=0,t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        legend_title_text='Node Categories',
        legend=dict(x=1, y=0, xanchor='right', yanchor='bottom')
    )

    return fig


## APP LAYOUT SET UP

In [175]:
from dash import Dash
import dash_bootstrap_components as dbc

external_stylesheets = [dbc.themes.BOOTSTRAP]

app = Dash(__name__,
           external_stylesheets=external_stylesheets,
           suppress_callback_exceptions=True,  
           prevent_initial_callbacks=True) 


#Creating navigation bar : 
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Home", href="/")),
        dbc.NavItem(dbc.NavLink("Analytics", href="/analytics")),
    ],
    brand="MOMENTUM, VOLATILITY & CORRELATIONS IN FINANCIAL MARKETS - AN EXPLORATIVE (DEMO) DASHBOARD",
    brand_href="/",
    color="primary",
    dark=True,
)


# Define the app layout with different pages
app.layout = html.Div([
    dcc.Store(id='session-data'),
    dcc.Location(id='url', refresh=False),
    navbar,
    html.Div(id='page-content'), 
])

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/analytics':
        return layout_analytics
    elif pathname == '/portfolio':
        return layout_portfolio
    elif pathname == '/':
        return layout_home
    else:
        return html.Div([
            html.H1('404 Error'),
            html.P('Page not found: the pathname was {}'.format(pathname))
        ], style={'textAlign': 'center'})


# LAYOUTS

### Home

In [176]:
from dash import dcc, html
import dash_bootstrap_components as dbc

layout_home = html.Div([
#     dcc.Store(id='session-data', data={})
    html.H2("Home Page", style={'textAlign': 'left', 'color':'white'}),
    html.H2("Construct your custom Dataframe:", style={'textAlign': 'center', 'color':'white'}),
    dbc.Container([
        dbc.Row([
            dbc.Col(html.Div([
                html.H3("Enter Stock Ticker:", style={'textAlign': 'center', 'color' : 'blue'}),
                dbc.Input(id='ticker-input', placeholder='Enter ticker, e.g., AAPL', type='text', value='AAPL', style={'margin': '30px 0'})
            ], style={'backgroundColor': 'white', 'padding': '10px', 'borderRadius': '5px', 'boxShadow': '0 4px 8px 0 rgba(0, 0, 0, 0.2)'}), width=12)
        ], justify='center', className="mb-4", style={'paddingTop': '10px'}),

        dbc.Row([
            dbc.Col(
                dcc.Dropdown(
                    id='asset-class-dropdown',
                    options=[
                        {'label': 'Equity', 'value': 'Equity'},
                        {'label': 'Forex', 'value': 'Forex'},
                        {'label': 'Fixed Income', 'value': 'Fixed Income'},
                        {'label': 'Commodities', 'value': 'Commodities'}, 
                        {'label' : 'Index' , 'value' : 'Index'},
                    ], placeholder="Select asset class", value="Equity", 
                    style={'backgroundColor': '#f0f0f0', 'color': 'black'}), width=4),
            dbc.Col(
                dbc.Input(id='start-date-input', placeholder='Start Date (YYYY-MM-DD)', type='text', value='2000-01-01'), width=4),
            dbc.Col(
                dbc.Input(id='end-date-input', placeholder='End Date (YYYY-MM-DD)', type='text', value=dt.today().strftime('%Y-%m-%d')), width=4)
        ], justify='center', className="mb-4"), 

        dbc.Row([
            dbc.Col([
                dbc.Checkbox(id='scale-checkbox', className='form-check-input'),
                html.Label('Scaled', htmlFor='scale-checkbox', className='form-check-label', style={'margin-left': '10px'})
            ], width=3, align='start'),
            dbc.Col(dbc.Button('Download Data', id='submit-button', color='danger', n_clicks=0, className='btn-lg'), width=3, align='center'),
            dbc.Col(dbc.Button('Reset Data', id='reset-button', color='primary', n_clicks=0, className='btn-lg'), width=3, align='end')
        ], justify='center', className="mb-3"),

        dbc.Row([
            dbc.Col(html.Div(id='output-container', style={'color': 'white'}), width=12)
        ], justify='center', className="mb-3")
    ], style={'height': '100vh', 'backgroundColor': '#000000', 'color': 'white'})
], style={'backgroundColor': '#000000'})


### Analytics 

In [177]:
from dash import dcc, html

layout_analytics = html.Div([
    html.H1("Directional Momentum Visualization", style={'textAlign': 'center'}),
    html.H2("Network Graph Visualization of Asset Prices Correlations", style={'textAlign': 'center'}),
    dcc.Store(id='store-corr-threshold', storage_type='session'),  # Store for correlation threshold
    dcc.Store(id='store-span', storage_type='session'),  # Store for span
    dcc.Store(id='store-selected-date', storage_type='session'),  # Store for selected date
    dcc.Store(id='net-graph', storage_type='session'),
    dcc.Store(id='heatmap-relative', storage_type='session'),
    dcc.Store(id='heatmap-directional', storage_type='session'),

    html.Div([
        html.Div([
            html.Label("Correlation Threshold", style={'textAlign': 'center'}),
            dcc.Slider(
                id='corr-threshold-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.5,  # Default value
                marks={str(i/10): str(i / 10) for i in range(0, 11)}, 
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '20px'}),

        html.Div([
            html.Label("Span", style={'textAlign': 'center'}),
            dcc.Input(
                id='span-input',
                type='number',
                value=5,  
                min=1,  
                max=100,  
                step=1  
            )
        ], style={'width': '15%', 'display': 'inline-block', 'padding': '20px'}),

        html.Div([
            dcc.DatePickerSingle(
                id='date-picker',
                min_date_allowed='start',
                max_date_allowed='end',
                initial_visible_month='start',
                date=str('start')  # Set initial date
            )
        ], style={'width': '30%', 'display': 'inline-block', 'textAlign': 'right', 'float': 'right', 'padding': '20px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    

    dcc.Graph(id='network-graph'),
    html.Div([
        html.Div([
            html.H2("Relative Strength Heatmap", style={'textAlign': 'center'}),
            dcc.Graph(id='heatmap-relative'),
        ], style={'width': '50%', 'display': 'inline-block'}),

        html.Div([
            html.H2("Directional Strength Heatmap", style={'textAlign': 'center'}),
            dcc.Graph(id='heatmap-directional'),
        ], style={'width': '50%', 'display': 'inline-block'}),
    ], style={'display': 'flex'}),  # This container will hold both heatmaps side by side
    
    html.Div(id='error-message', style={'color': 'red', 'fontWeight': 'bold'})  # Error message div
], style={'padding': '20px'})


### Portfolio 

In [178]:
layout_portfolio = html.Div([
    html.H1('Portfolio Page'),
    # Further components and layout details for the Home page
])

# CALLBACKS

### Home 

In [179]:
@app.callback(
    [Output('output-container', 'children'),
     Output('session-data', 'data')],
    [Input('submit-button', 'n_clicks'),
     Input('reset-button', 'n_clicks'),
     Input('scale-checkbox', 'value'),
     Input('session-data', 'data'),
     Input('url', 'pathname')],  # Listen to URL changes
    [State('ticker-input', 'value'),
     State('asset-class-dropdown', 'value'),
#      State('sector-dropdown', 'value'),
     State('start-date-input', 'value'),
     State('end-date-input', 'value')]
)
def update_or_reload_data(submit_n_clicks, reset_n_clicks, scale, jsonified_data, pathname,
                          ticker, asset_class, start_date, end_date):
    global main_df
    global initializer
    global list_tickers, dict_classes, dict_sectors, dict_commo
    key = 0
    triggered_id = callback_context.triggered[0]['prop_id'].split('.')[0]

    if initializer:
        initializer = False
        return html.Div("Empty Graph Data"), {'Price Plot' : None, 'start' : None, 'end' : None}
    
    if triggered_id == 'reset-button' and reset_n_clicks > 0:
        main_df = pd.DataFrame()
        reset_global_vars()
        return html.Div("Data has been reset"), {'Price Plot' : None, 'start' : None, 'end' : None}
    
    
    
    if triggered_id == 'submit-button' or triggered_id == 'scale-checkbox':
        try:
            df = data_loader(ticker, start=start_date, end=end_date)
            df.dropna(inplace=True)
            if df.empty:
                return html.Div("No data available for the selected ticker and date range."), {'Price Plot' : None, 'start' : None, 'end' : None}

            if main_df.empty:
                main_df = df
            else:
                prefix = ticker
                matching_columns = [col for col in main_df.columns if col.startswith(prefix)]
                main_df.drop(columns=matching_columns, inplace=True)
#                 main_df = pd.merge(main_df, df, left_index=True, right_index=True, how='inner')
                if len(df) <= len(main_df):
                    #Update the old df by merging with new df - INNER JOIN 
                    main_df = pd.merge(main_df, df, left_index=True, right_index=True, how='inner')
                else: 
                    data_loader_format_all(ticker, start_date, end_date)
        except Exception as e:
            return html.Div(f"Failed to load data for {ticker}: {str(e)}"), {'Price Plot' : None, 'start' : None, 'end' : None}  
    
    elif jsonified_data:
        if 'Price Plot' in jsonified_data and jsonified_data['Price Plot']:
            json_str = jsonified_data['Price Plot']
            try:
                json_dump = pd.read_json(json_str, orient='split')
                key = 1 
            except ValueError as e:
                print("Error loading JSON data:", e)
                return html.Div("Failed to load data."), {'Price Plot': None, 'start': None, 'end': None}

    # Subset DataFrame focusing on 'Adj Close'
    if key == 0:
        col_list = list(main_df.columns)
        adj_close_list = [item for item in col_list if "Adj Close" in item]
        sub_df = main_df[adj_close_list]
        sub_df.columns = [re.sub(r'^\s+|\s+$', '', col.split(":")[0]) for col in adj_close_list]
    else:
        col_list = list(json_dump.columns)
        adj_close_list = [item for item in col_list if "Adj Close" in item]
        sub_df = json_dump[adj_close_list]
        sub_df.columns = [re.sub(r'^\s+|\s+$', '', col.split(":")[0]) for col in adj_close_list]

    if scale and not sub_df.empty:
        sub_df = scaled_df(sub_df)

    # Plotting the graph
    fig = go.Figure()
    for ticker in sub_df.columns:
        fig.add_trace(go.Scatter(x=sub_df.index, y=sub_df[ticker], mode='lines', name=ticker))

    fig.update_layout(
        title='Adjusted Close Prices Over Time for selected tickers',
        xaxis_title='Date',
        yaxis_title='Adjusted Close Price',
        legend_title="Ticker"
    )

    start_time = dt.strptime(start_date, "%Y-%m-%d").date()
    end_time = dt.strptime(end_date, "%Y-%m-%d").date()

    #UPDATE TO GLOBAL VARIABLES
    list_tickers.append(ticker)
    list_tickers = list(set(list_tickers))

    dict_classes[asset_class].append(ticker)
    dict_classes[asset_class] = list(set(dict_classes[asset_class]))

#     if asset_class == 'Commodities':
#         dict_commo[asset_sector].append(ticker)
#         dict_commo[asset_sector] = list(set(dict_commo[asset_sector]))
#     if asset_class == 'Equity':
#         dict_sectors[asset_sector].append(ticker)
#         dict_sectors[asset_sector] = list(set(dict_sectors[asset_sector]))

    return dcc.Graph(figure=fig), {'Price Plot' : sub_df.to_json(date_format='iso', orient='split'), 
                                   'start' : start_time, 
                                   'end' : end_time}
    
#     return "Enter values and click 'Download Data' or 'Reset Data'.", {'Price Plot' : None, 'start' : None, 'end' : None} 





# @app.callback(
#     Output('sector-dropdown', 'options'),
#     [Input('asset-class-dropdown', 'value')]
# )
# def set_sectors_options(selected_asset_class):
#     if selected_asset_class == 'Equity':
#         return [{'label': 'Technology', 'value': 'Technology'},
#                 {'label': 'Consumer', 'value': 'Consumer'},
#                 {'label': 'Utilities', 'value': 'Uitilies'},
#                 {'label': 'Industrial', 'value': 'Consumer'},
#                 {'label': 'Financials', 'value': 'Financials'},
#                 {'label': 'Other', 'value': 'Other'},]
#     elif selected_asset_class == 'Commodities':
#         return [{'label': 'Agriculture', 'value': 'Agriculture'},
#                 {'label': 'Precious Metals', 'value': 'Precious Metals'},
#                 {'label': 'Industrials', 'value': 'Industrials'},
#                 {'label': 'Energy', 'value': 'Energy'}]
#     else:
#         return [{'label' : 'N/A', 'value' : 'N/A'}]
    

    
# @app.callback(
#     Output('sector-dropdown', 'value'),
#     [Input('asset-class-dropdown', 'value')]
# )
# def set_default_sector(selected_asset_class):
#     if selected_asset_class == 'Equity':
#         return 'Technology'  
#     elif selected_asset_class == 'Commodities':
#         return 'Agriculture'  
#     return None  


### Analytics 

In [180]:
@app.callback(
    Output('span-input', 'value'),
    [Input('date-picker', 'date'),
     Input('span-input', 'value')]
)
def debugging(ref_date, span):
    
    #First : Check if Span input is indeed a positive int
    assert isinstance(span, int)
    assert span >= 0, "span must be a positive integer"
    
    #Second : Debugging the Ref_Date 
    if ref_date not in main_df.index:
        print('Ref_date not in df.index')
        ref_date = main_df.index[0]
    else: 
        ref_date = ref_date 
    
    #Third : Debugging the Span  
    #Retrieve position of the current date
    position = main_df.index.get_loc(ref_date) + 1
    if 2*span > position: #because we need 2 periods of length span 
        print('Span too high compared to index position. Rescaling')
        span = position // 2

    print(f'Select position : {main_df.index[span]} minimum for a span window of {span}.')

    return span


@app.callback(
    [Output('network-graph', 'figure'),
     Output('heatmap-relative', 'figure'),
     Output('heatmap-directional', 'figure'),
     Output('store-corr-threshold', 'data'),
     Output('store-span', 'data'),
     Output('store-selected-date', 'data'),
     Output('error-message', 'children')],
    [Input('date-picker', 'date'),
     Input('corr-threshold-slider', 'value'), 
     Input('span-input', 'value')]
)
def update_graph(selected_date, corr_threshold, span):
    global main_df
    
    #Only selecting the columns which are named as "Ticker : Log-Returns" from main_df
    suffix = 'Log-Returns'
    subset_cols = [col for col in list(main_df.columns) if col.endswith(suffix)]
    subset_df = main_df[subset_cols]
    subset_df.columns = list_tickers
    
    
    span = debugging(selected_date, span)
    
    if selected_date is not None and corr_threshold is not None:
        # Check if selected_date is valid using rolling_corr or equivalent function
        corr_result = rolling_corr(subset_df, selected_date, span)  
        if isinstance(corr_result, str) and corr_result == "Date not found":
            return dash.no_update, dash.no_update, dash.no_update, corr_threshold, span, selected_date, "Error: Selected date not found in the dataset."
        else:
            try: 
                network_fig = graph_net(subset_df, selected_date, corr_threshold=corr_threshold, span=span)
                heatmap_relative_graph, heatmap_directional_graph = rolling_corr_difference(subset_df, selected_date, span=span)
                return network_fig, heatmap_relative_graph, heatmap_directional_graph, corr_threshold, span, selected_date, ""
            
            except Exception as e:
                return dash.no_update, dash.no_update, dash.no_update, corr_threshold, span, selected_date, f"Error: {str(e)}"
    else:
        return dash.no_update, dash.no_update, dash.no_update, corr_threshold, span, selected_date, "" 

    
@app.callback(
    [Output('date-picker', 'min_date_allowed'),
     Output('date-picker', 'max_date_allowed'),
     Output('date-picker', 'initial_visible_month'),
     Output('date-picker', 'date')],
    [Input('session-data', 'data')],
)
def update_date_picker(session_data):
    if session_data:
        min_date = session_data.get('start')
        max_date = session_data.get('end')
        return min_date, max_date, min_date, min_date
    # Default to some range if no data is found
    default_date = datetime.datetime.today().date()
    return default_date, default_date, default_date, default_date



# @app.callback(
#     [Output('store-corr-threshold', 'data'),
#      Output('store-span', 'data'),
#      Output('store-selected-date', 'data')],
#     [Input('corr-threshold-slider', 'value'),
#      Input('span-input', 'value'),
#      Input('date-picker', 'date')],
#     prevent_initial_call=True
# )
# def store_values(corr_threshold, span, selected_date):
#     return corr_threshold, span, selected_date


# # Callback to update UI elements based on stored data if necessary
# @app.callback(
#     [Output('corr-threshold-slider', 'value'),
#      Output('span-input', 'value'),
#      Output('date-picker', 'date')],
#     [Input('store-corr-threshold', 'data'),
#      Input('store-span', 'data'),
#      Input('store-selected-date', 'data')],
#     prevent_initial_call=True
# )
# def update_ui_from_store(corr_threshold, span, selected_date):
#     return corr_threshold, span, selected_date

### Portfolio


# Index

In [181]:
import webbrowser

# Run the app
port = 3009

# Open a web browser tab using the specified port
def open_browser():
      webbrowser.open_new_tab(f'http://127.0.0.1:{port}')

if __name__ == '__main__':
    # Use the threading module to open a web browser tab
    # This prevents blocking the execution of the app
    from threading import Timer
    Timer(1, open_browser).start()  # Wait 1 second before opening the tab
    
    app.run_server(debug=True, port=port)

In [182]:
main_df

""


In [183]:
dict_sectors

{'Financials': [],
 'Technology': [],
 'Industrial': [],
 'Consumer': [],
 'Utilities': [],
 'Other': []}

In [184]:
dict_classes

{'Equity': [], 'Commodities': [], 'Fixed Income': [], 'Forex': [], 'Index': []}

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Ref_date not in df.index
Span too high compared to index position. Rescaling
Select position : 2014-09-17 00:00:00 minimum for a span window of 0.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

invalid value encountered in log

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[************

Ref_date not in df.index
Span too high compared to index position. Rescaling
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Ref_date not in df.index
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Error due to wrong date/span input: '2019-06-01', Date : 2019-06-01, Span : 0.
Recall date range of input dataframe: (Timestamp('2019-06-04 00:00:00'), Timestamp('2021-06-30 00:00:00'))
Error due to wrong date/span input: '2019-06-01', Date : 2019-06-01, Span : 0.
Recall date range of input dataframe: (Timestamp('2019-06-04 00:00:00'), Timestamp('2021-06-30 00:00:00'))
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
nan nan
TSLA TSLA
nan nan
TSLA CL=F
nan nan
TSLA JPM
nan nan
TSLA DX=F
nan nan
TSLA ^RUT
nan nan
TSLA IAU
nan nan
TSLA BTC-USD
nan nan
TSLA GSG
nan nan
TSLA TLT
nan nan
TSLA ^GDAXI
nan nan
TSLA NVDA
nan nan
TSLA ^FTSE
nan nan
TSLA AAPL
nan nan
TS

...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

invalid value encountered in log

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[************

Ref_date not in df.index
Span too high compared to index position. Rescaling
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Ref_date not in df.index
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Error due to wrong date/span input: '2019-06-01', Date : 2019-06-01, Span : 0.
Recall date range of input dataframe: (Timestamp('2019-06-04 00:00:00'), Timestamp('2021-06-30 00:00:00'))
Error due to wrong date/span input: '2019-06-01', Date : 2019-06-01, Span : 0.
Recall date range of input dataframe: (Timestamp('2019-06-04 00:00:00'), Timestamp('2021-06-30 00:00:00'))
Ref_date not in df.index
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Ref_date not in df.index
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
Select position : 2019-06-04 00:00:00 minimum for a span window of 0.
nan nan
TSLA TSLA
nan nan
TSLA CL=F
nan nan
TSLA JPM
nan

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

invalid value encountered in log

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[************

Ref_date not in df.index
Span too high compared to index position. Rescaling
Select position : 2014-09-17 00:00:00 minimum for a span window of 0.
Ref_date not in df.index
Select position : 2014-09-17 00:00:00 minimum for a span window of 0.
Error due to wrong date/span input: '2000-01-01', Date : 2000-01-01, Span : 0.
Recall date range of input dataframe: (Timestamp('2014-09-17 00:00:00'), Timestamp('2024-04-12 00:00:00'))
Error due to wrong date/span input: '2000-01-01', Date : 2000-01-01, Span : 0.
Recall date range of input dataframe: (Timestamp('2014-09-17 00:00:00'), Timestamp('2024-04-12 00:00:00'))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

invalid value encountered in log

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[************